# Solutions to Project Euler 21 - 30

In [1]:
%load_ext fetch_euler_problem

## Problem 21

[Amicable numbers](https://projecteuler.net/problem=21)

> <p>Let d(<i>n</i>) be defined as the sum of proper divisors of <i>n</i> (numbers less than <i>n</i> which divide evenly into <i>n</i>).<br/>If d(<i>a</i>) = <i>b</i> and d(<i>b</i>) = <i>a</i>, where <i>a</i> ≠ <i>b</i>, then <i>a</i> and <i>b</i> are an amicable pair and each of <i>a</i> and <i>b</i> are called amicable numbers.</p>
> <p>For example, the proper divisors of 220 are 1, 2, 4, 5, 10, 11, 20, 22, 44, 55 and 110; therefore d(220) = 284. The proper divisors of 284 are 1, 2, 4, 71 and 142; so d(284) = 220.</p>
> <p>Evaluate the sum of all the amicable numbers under 10000.</p>

In [2]:
import math


def proper_divisor_sum(n: int) -> int:
    """
    >>> proper_divisor_sum(220)
    284
    
    Note: `sympy.divisor_sigma(n, k)` gives more divisor functions.
    """
    assert n >= 1

    divisors = set()
    ub = int(math.sqrt(n))
    for i in range(1, ub + 1):
        if n % i == 0:
            divisors.add(i)
            divisors.add(n // i)
    return sum(divisors) - n


def is_amicable(a: int) -> bool:
    """
    >>> is_amicable(220)
    True
    """
    assert a > 1
    b = proper_divisor_sum(a)
    return a != b and a == proper_divisor_sum(b)


def prob021(n=10000):
    return sum(i for i in range(2, n + 1) if is_amicable(i))


In [3]:
%time prob021()

CPU times: user 140 ms, sys: 148 µs, total: 140 ms
Wall time: 138 ms


31626

## Problem 22

[Names scores](https://projecteuler.net/problem=22)

> <p>Using <a href="project/resources/p022_names.txt">names.txt</a> (right click and 'Save Link/Target As...'), a 46K text file containing over five-thousand first names, begin by sorting it into alphabetical order. Then working out the alphabetical value for each name, multiply this value by its alphabetical position in the list to obtain a name score.</p>
> <p>For example, when the list is sorted into alphabetical order, COLIN, which is worth 3 + 15 + 12 + 9 + 14 = 53, is the 938th name in the list. So, COLIN would obtain a score of 938 × 53 = 49714.</p>
> <p>What is the total of all the name scores in the file?</p>

In [4]:
import urllib.request


def worth(name: str) -> int:
    """
    >>> worth('COLIN')
    53
    """
    return sum(ord(c) - ord("A") + 1 for c in name.upper())


def prob022():
    URL022 = "https://projecteuler.net/project/resources/p022_names.txt"
    with urllib.request.urlopen(URL022) as f:
        raw = f.read()

    s = raw.decode()
    names = s.replace('"', "").split(",")
    return sum(i * worth(name) for i, name in enumerate(names, 1))


In [5]:
prob022()

850081394

## Problem 23

[Non-abundant sums](https://projecteuler.net/problem=23)

> <p>A perfect number is a number for which the sum of its proper divisors is exactly equal to the number. For example, the sum of the proper divisors of 28 would be 1 + 2 + 4 + 7 + 14 = 28, which means that 28 is a perfect number.</p>
> <p>A number <var>n</var> is called deficient if the sum of its proper divisors is less than <var>n</var> and it is called abundant if this sum exceeds <var>n</var>.</p>
> <p>As 12 is the smallest abundant number, 1 + 2 + 3 + 4 + 6 = 16, the smallest number that can be written as the sum of two abundant numbers is 24. By mathematical analysis, it can be shown that all integers greater than 28123 can be written as the sum of two abundant numbers. However, this upper limit cannot be reduced any further by analysis even though it is known that the greatest number that cannot be expressed as the sum of two abundant numbers is less than this limit.</p>
> <p>Find the sum of all the positive integers which cannot be written as the sum of two abundant numbers.</p>

In [6]:
import math


# Reproduced from Problem 21
def proper_divisor_sum(n: int) -> int:
    """
    >>> proper_divisor_sum(220)
    284
    
    Note: `sympy.divisor_sigma(n, k)` gives more divisor functions.
    """
    assert n >= 1

    divisors = set()
    ub = int(math.sqrt(n))
    for i in range(1, ub + 1):
        if n % i == 0:
            divisors.add(i)
            divisors.add(n // i)
    return sum(divisors) - n


def is_abundant(n: int) -> bool:
    return proper_divisor_sum(n) > n


def prob023():
    ub = 28123
    abundant_numbers = {i for i in range(1, ub + 1) if is_abundant(i)}

    def _can_be_sum_of_two_abundant(n):
        return any(
            i in abundant_numbers and (n - i) in abundant_numbers
            for i in range(1, n // 2 + 1)
        )

    res = {n for n in range(1, ub + 1) if not _can_be_sum_of_two_abundant(n)}
    return sum(res)


In [7]:
%time prob023()

CPU times: user 1.92 s, sys: 1.93 ms, total: 1.92 s
Wall time: 1.92 s


4179871

## Problem 24

[Lexicographic permutations](https://projecteuler.net/problem=24)

> <p>A permutation is an ordered arrangement of objects. For example, 3124 is one possible permutation of the digits 1, 2, 3 and 4. If all of the permutations are listed numerically or alphabetically, we call it lexicographic order. The lexicographic permutations of 0, 1 and 2 are:</p>
> <p style="text-align:center;">012   021   102   120   201   210</p>
> <p>What is the millionth lexicographic permutation of the digits 0, 1, 2, 3, 4, 5, 6, 7, 8 and 9?</p>

In [8]:
import functools
import operator


def factorial(n:int) -> int:
    return functools.reduce(operator.mul, range(1, n+1), 1)


def prob024(digits=tuple(range(10)), rank=1000000):
    n = len(digits)
    d = list(digits)
    result = []
    for i in range(n-1, -1, -1):
        idx, rank = divmod(rank, factorial(i))
        result.append(d[idx])
        del d[idx]

    return ''.join(str(i) for i in result)

In [9]:
%time prob024()

CPU times: user 30 µs, sys: 2 µs, total: 32 µs
Wall time: 36.2 µs


'2783915604'

## Problem 25

[1000-digit Fibonacci number](https://projecteuler.net/problem=25)

> The Fibonacci sequence is defined by the recurrence relation:
>
> $$ F_n = F_{n−1} + F_{n−2}, \quad \text{ where } \quad F_1 = 1 \quad \text{ and } \quad F_2 = 1. $$
>
> Hence the first 12 terms will be:
> 
> $$ 
\begin{align*}
    F_1 &= 1  \\
    F_2 &= 1  \\
    F_3 &= 2  \\
    F_4 &= 3  \\
    F_5 &= 5   \\
    F_6 &= 8   \\
    F_7 &= 13  \\
    F_8 &= 21  \\
    F_9 &= 34  \\
    F_{10} &= 55 \\
    F_{11} &= 89 \\
    F_{12} &= 144
\end{align*}
$$
>
>  The 12th term, $F_{12}$, is the first term to contain three digits.
> 
> What is the index of the first term in the Fibonacci sequence to contain $1000$ digits?

In [10]:
import itertools

# Reproduced from Problem 2
def fibonacci():
    i, j = 0, 1
    yield j
    while True:
        (i, j) = (j, i + j)
        yield j


def star(f):
    """
    This enables tuple unpacking that was removed in python 3.
    http://stackoverflow.com/questions/21892989/

    >>> points = [(1, 2), (-1, 0)]
    >>> min(points, key=star(lambda x,y: x*x + y*y))
    (-1, 0)
    """

    def _g(args):
        return f(*args)

    return _g


def digit_length(n: int) -> int:
    return len(str(n))


def prob025(digits=1000):
    """
    >>> prob025(digits=3)
    12
    """
    fibs = fibonacci()
    res, _ = next(
        itertools.dropwhile(
            star(lambda _, v: digit_length(v) < digits), enumerate(fibs, 1)
        )
    )
    return res


In [11]:
prob025()

4782

## Problem 26

[Reciprocal cycles](https://projecteuler.net/problem=26)

> <p>A unit fraction contains 1 in the numerator. The decimal representation of the unit fractions with denominators 2 to 10 are given:</p>
> <blockquote><table><tr><td><sup>1</sup>/<sub>2</sub></td><td>= </td><td>0.5</td></tr><tr><td><sup>1</sup>/<sub>3</sub></td><td>= </td><td>0.(3)</td></tr><tr><td><sup>1</sup>/<sub>4</sub></td><td>= </td><td>0.25</td></tr><tr><td><sup>1</sup>/<sub>5</sub></td><td>= </td><td>0.2</td></tr><tr><td><sup>1</sup>/<sub>6</sub></td><td>= </td><td>0.1(6)</td></tr><tr><td><sup>1</sup>/<sub>7</sub></td><td>= </td><td>0.(142857)</td></tr><tr><td><sup>1</sup>/<sub>8</sub></td><td>= </td><td>0.125</td></tr><tr><td><sup>1</sup>/<sub>9</sub></td><td>= </td><td>0.(1)</td></tr><tr><td><sup>1</sup>/<sub>10</sub></td><td>= </td><td>0.1</td></tr></table></blockquote>
> <p>Where 0.1(6) means 0.166666..., and has a 1-digit recurring cycle. It can be seen that <sup>1</sup>/<sub>7</sub> has a 6-digit recurring cycle.</p>
> <p>Find the value of <i>d</i> &lt; 1000 for which <sup>1</sup>/<sub><i>d</i></sub> contains the longest recurring cycle in its decimal fraction part.</p>

Only numbers composed purely of primes 2 and/or 5 are divisible. Length $x$ of the recurring cycle of $1/p$ is determined by the minimum integer $x$ satisfing 

$$ 10^x - 1 \equiv 0 \pmod{p} $$



For example with $Z = 1/7 = 0.(142857)$, 
$$ 
\begin{align*}
    10^6 Z &= 142857.(142857) \\
         Z &=  0.(142857) 
\end{align*}
$$

Substracting one equation from the other, we can cancell out the decimal fraction part, giving

$$  Z = \frac{142857}{10^6 - 1}. $$

In [12]:
import itertools as it


def psudo_primes():
    """
    psudo-prime numbers that exclude 2, 5 or multiples of them
    """
    yield 3
    x = 5
    while True:
        if x % 5 > 0:
            yield x
        x += 2
        if x % 5 > 0:
            yield x
        x += 4


def recurring_cycle_length(p):
    assert p > 0
    assert p % 2 > 0
    assert p % 5 > 0
    return next(i for i in it.count(1) if (10 ** i - 1) % p == 0)


def prob027(dmax=1000):
    """
    >>> prob027(dmax=10)
    7
    """
    prime_like = it.takewhile(lambda p: p < dmax, psudo_primes())
    return max(prime_like, key=recurring_cycle_length)


In [13]:
%timeit prob027()

114 ms ± 18.7 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


## Problem 27

[Quadratic primes](https://projecteuler.net/problem=27)

> <p>Euler discovered the remarkable quadratic formula:</p>
> <p style="text-align:center;">$n^2 + n + 41$</p>
> <p>It turns out that the formula will produce 40 primes for the consecutive integer values $0 \le n \le 39$. However, when $n = 40, 40^2 + 40 + 41 = 40(40 + 1) + 41$ is divisible by 41, and certainly when $n = 41, 41^2 + 41 + 41$ is clearly divisible by 41.</p>
> <p>The incredible formula $n^2 - 79n + 1601$ was discovered, which produces 80 primes for the consecutive values $0 \le n \le 79$. The product of the coefficients, −79 and 1601, is −126479.</p>
> <p>Considering quadratics of the form:</p>
> <blockquote>$n^2 + an + b$, where $|a| < 1000$ and $|b| \le 1000$<br/><br/><div>where $|n|$ is the modulus/absolute value of $n$<br/>e.g. $|11| = 11$ and $|-4| = 4$</div></blockquote>
> <p>Find the product of the coefficients, $a$ and $b$, for the quadratic expression that produces the maximum number of primes for consecutive values of $n$, starting with $n = 0$.</p>

Let $f(n) = n^2 + a n + b$. When $n=0$, $f(0) = b$ must be prime. When $n=1$, $f(1) = 1 + a + b$ must be prime. So $a$ must be odd unless $b = \pm 2$; note $b = \pm 2$ cannot make $f(2) = 4 + 2a +b$ prime for any choice of $a$. Therefore, we restrict $a$ and $b$ by

- $b$ is prime (positive)
- $a$ is even
- $a + b > 0$

In [14]:
import itertools as it
import math


def psudo_primes():
    yield 3
    x = 5
    while True:
        yield x
        x += 2
        yield x
        x += 4


def is_prime(n: int) -> bool:
    """
    >>> is_prime(19)
    True
    """
    if n < 2:
        return False
    elif n % 2 == 0:
        return False
    else:
        candidates = it.takewhile(lambda x: x <= int(math.sqrt(n)), psudo_primes())
        return all(n % p > 0 for p in candidates)


def consecutive_prime_len(a, b):
    """
    >>> consecutive_prime_len(a=1, b=41)
    40
    """

    def quadratic(n):
        return (n + a) * n + b

    iterable = it.takewhile(lambda i: is_prime(quadratic(i)), it.count())
    return sum(1 for _ in iterable)


def prob027():
    ps = it.takewhile(lambda x: x < 1000, psudo_primes())
    (size, a, b) = max(
        (consecutive_prime_len(a, b), a, b)
        for b in ps
        for a in range(-999, 1000, 2)
        if a + b > 0
    )
    res = (f"With (a, b) = ({a}, {b}), one gets {size} consecutive primes: "
           f"a x b = {a * b}")
    print(res)


In [15]:
%time prob027()

With (a, b) = (-61, 971), one gets 71 consecutive primes: a x b = -59231
CPU times: user 2.92 s, sys: 14 µs, total: 2.92 s
Wall time: 2.91 s


## Problem 28

[Number spiral diagonals](https://projecteuler.net/problem=28)

> <p>Starting with the number 1 and moving to the right in a clockwise direction a 5 by 5 spiral is formed as follows:</p>
> <p style="text-align:center;font-family:'courier new';"><span style="color:#ff0000;font-family:'courier new';"><b>21</b></span> 22 23 24 <span style="color:#ff0000;font-family:'courier new';"><b>25</b></span><br/>20  <span style="color:#ff0000;font-family:'courier new';"><b>7</b></span>  8  <span style="color:#ff0000;font-family:'courier new';"><b>9</b></span> 10<br/>19  6  <span style="color:#ff0000;font-family:'courier new';"><b>1</b></span>  2 11<br/>18  <span style="color:#ff0000;font-family:'courier new';"><b>5</b></span>  4  <span style="color:#ff0000;font-family:'courier new';"><b>3</b></span> 12<br/><span style="color:#ff0000;font-family:'courier new';"><b>17</b></span> 16 15 14 <span style="color:#ff0000;font-family:'courier new';"><b>13</b></span></p>
> <p>It can be verified that the sum of the numbers on the diagonals is 101.</p>
> <p>What is the sum of the numbers on the diagonals in a 1001 by 1001 spiral formed in the same way?</p>

From observation I see increments are 2, 2, 2, 2, 4, 4, 4, 4, 6, 6, 6, 6, ... starting form 1.

In [16]:
import itertools as it

def increments():
    yield 1
    x = 2
    while True:
        for _ in range(4):
            yield x
        x += 2

def prob028(n=1001):
    """
    >>> prob028(5)
    101
    """
    incs = it.takewhile(lambda x: x <= n - 1, increments())
    vals = it.accumulate(incs)
    return sum(vals)

In [17]:
%time prob028()

CPU times: user 1.14 ms, sys: 1 µs, total: 1.14 ms
Wall time: 1.18 ms


669171001

## Problem 29

[Distinct powers](https://projecteuler.net/problem=29)

> <p>Consider all integer combinations of <i>a</i><sup><i>b</i></sup> for 2 ≤ <i>a</i> ≤ 5 and 2 ≤ <i>b</i> ≤ 5:</p>
> <blockquote>2<sup>2</sup>=4, 2<sup>3</sup>=8, 2<sup>4</sup>=16, 2<sup>5</sup>=32<br/>3<sup>2</sup>=9, 3<sup>3</sup>=27, 3<sup>4</sup>=81, 3<sup>5</sup>=243<br/>4<sup>2</sup>=16, 4<sup>3</sup>=64, 4<sup>4</sup>=256, 4<sup>5</sup>=1024<br/>5<sup>2</sup>=25, 5<sup>3</sup>=125, 5<sup>4</sup>=625, 5<sup>5</sup>=3125<br/></blockquote>
> <p>If they are then placed in numerical order, with any repeats removed, we get the following sequence of 15 distinct terms:</p>
> <p style="text-align:center;">4, 8, 9, 16, 25, 27, 32, 64, 81, 125, 243, 256, 625, 1024, 3125</p>
> <p>How many distinct terms are in the sequence generated by <i>a</i><sup><i>b</i></sup> for 2 ≤ <i>a</i> ≤ 100 and 2 ≤ <i>b</i> ≤ 100?</p>

Brute force approach is good enough. Use `set` to handle duplicates.

In [18]:
def prob029():
    return len({a**b for a in range(2, 101) for b in range(2, 101)})

In [19]:
%time prob029()

CPU times: user 9 ms, sys: 0 ns, total: 9 ms
Wall time: 8.95 ms


9183

## Problem 30

[Digit fifth powers](https://projecteuler.net/problem=30)

> <p>Surprisingly there are only three numbers that can be written as the sum of fourth powers of their digits:</p>
> <blockquote>1634 = 1<sup>4</sup> + 6<sup>4</sup> + 3<sup>4</sup> + 4<sup>4</sup><br/>8208 = 8<sup>4</sup> + 2<sup>4</sup> + 0<sup>4</sup> + 8<sup>4</sup><br/>9474 = 9<sup>4</sup> + 4<sup>4</sup> + 7<sup>4</sup> + 4<sup>4</sup></blockquote>
> <p class="info">As 1 = 1<sup>4</sup> is not a sum it is not included.</p>
> <p>The sum of these numbers is 1634 + 8208 + 9474 = 19316.</p>
> <p>Find the sum of all the numbers that can be written as the sum of fifth powers of their digits.</p>

Upper limit exists. A $k$-digit number $x$ is in range $10^{k-1} \leq x < 10^k$, and the maximum possible sum of fifth powers of digits is $9^5 k$. So, $9^5 k < 10^{k-1} < x$ and equality never holds for $k \geq 7$. With $k=6$, the maximum possible sum of fifth powers of digits is $6 \times 9^5 = 354294$. So choose this number as an upper bound.

In [20]:
def integerdigits(n):
    """Construct list of decimal digits from the integer n

    >>> integerdigits(235)
    [2, 3, 5]
    """
    return [int(i) for i in str(n)]


def prob030(pow=5):
    """
    >>> prob030(pow=4)
    19316
    """
    ub = 6 * 9**pow
    return sum(n for n in range(2, ub) if sum(d**pow for d in integerdigits(n)) == n)

In [21]:
%time prob030()

CPU times: user 1.65 s, sys: 13.6 ms, total: 1.67 s
Wall time: 1.67 s


443839

## Doctests

In [22]:
import doctest
doctest.testmod()

TestResults(failed=0, attempted=11)